In [4]:
import os
import zipfile

# Dataset Extraction
dataset_path = r"C:\\Users\\HP\\Downloads\\archive (1).zip"
output_folder = "yoga_poses"

# Check if the output folder already exists
if not os.path.exists(output_folder):
    try:
        with zipfile.ZipFile(dataset_path, 'r') as zip_ref:
            zip_ref.extractall(output_folder)
            print(f"Dataset extracted to {output_folder}")
    except Exception as e:
        print(f"Error during extraction: {e}")
else:
    print(f"Folder {output_folder} already exists.")

# Check the directory path after extraction
dataset_dir = os.path.join(output_folder, "DATASET\\TEST")
if os.path.exists(dataset_dir):
    print(f"Dataset directory found: {dataset_dir}")
else:
    print(f"Dataset directory not found. Check the folder structure inside {output_folder}")


Folder yoga_poses already exists.
Dataset directory found: yoga_poses\DATASET\TEST


In [5]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models

# Parameters
img_size = 128
batch_size = 32
threshold = 60.0

# Classes
classes = [d for d in os.listdir(dataset_dir) if os.path.isdir(os.path.join(dataset_dir, d))]
print(f"Classes: {classes}")

# Data Augmentation
datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    validation_split=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

# Data Generators
train_data = datagen.flow_from_directory(
    dataset_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

val_data = datagen.flow_from_directory(
    dataset_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

# Model Definition
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_size, img_size, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(len(classes), activation='softmax')
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Model Training
epochs = 10
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=epochs
)

# Save Model
model_weights_path = "model_weights.weights.h5"
model.save_weights(model_weights_path)
print("Model training completed and weights saved.")


Classes: ['downdog', 'goddess', 'plank', 'tree', 'warrior2']
Found 378 images belonging to 5 classes.
Found 92 images belonging to 5 classes.


c:\Users\HP\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 15s 1s/step - accuracy: 0.1845 - loss: 1.9406 - val_accuracy: 0.2283 - val_loss: 1.5437
Epoch 2/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 10s 880ms/step - accuracy: 0.3089 - loss: 1.5404 - val_accuracy: 0.5543 - val_loss: 1.2249
Epoch 3/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 10s 843ms/step - accuracy: 0.5801 - loss: 1.1618 - val_accuracy: 0.6848 - val_loss: 0.8288
Epoch 4/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 9s 777ms/step - accuracy: 0.6397 - loss: 1.0185 - val_accuracy: 0.7283 - val_loss: 0.6920
Epoch 5/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 9s 794ms/step - accuracy: 0.6537 - loss: 0.8974 - val_accuracy: 0.7609 - val_loss: 0.7208
Epoch 6/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 9s 773ms/step - accuracy: 0.7656 - loss: 0.7171 - val_accuracy: 0.7717 - val_loss: 0.6383
Epoch 7/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 9s 765ms/step - accuracy: 0.7960 - loss: 0.6640 - val_accuracy: 0.8043 - val_loss: 0.5349
Epoch 8/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 9s 768ms/step - accuracy: 0.8157 - loss: 0.4756 - val_accuracy: 0.

In [6]:
import numpy as np
from PIL import Image

# Helper Functions
def analyze_image(image):
    corrections = []

    # Check resolution
    if image.size[0] < img_size or image.size[1] < img_size:
        corrections.append(f"Resolution too low. Image should be at least {img_size}x{img_size}.")

    # Check brightness
    grayscale_img = image.convert("L")
    avg_brightness = np.mean(grayscale_img)
    if avg_brightness < 50:
        corrections.append("Image is too dark. Increase brightness.")
    elif avg_brightness > 200:
        corrections.append("Image is too bright. Reduce brightness.")

    return corrections

def predict_pose_with_correction(image_path):
    img = tf.keras.utils.load_img(image_path, target_size=(img_size, img_size))
    img_array = tf.keras.utils.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0) / 255.0

    predictions = model.predict(img_array)
    confidence = np.max(predictions) * 100
    predicted_class = classes[np.argmax(predictions)] if confidence >= threshold else "Unknown"

    corrections = analyze_image(tf.keras.utils.load_img(image_path))
    return predicted_class, confidence, corrections


In [7]:
import cv2

# Real-time pose prediction function
def real_time_pose_prediction():
    cap = cv2.VideoCapture(0)
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Convert frame to PIL image for processing
        pil_image = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

        # Save frame temporarily to analyze
        temp_path = "temp_frame.jpg"
        pil_image.save(temp_path)

        # Predict pose and get corrections
        predicted_class, confidence, corrections = predict_pose_with_correction(temp_path)

        # Display results
        cv2.putText(frame, f"Pose: {predicted_class} ({confidence:.2f}%)",
                    (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        for i, correction in enumerate(corrections):
            cv2.putText(frame, correction, (10, 60 + i * 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)

        cv2.imshow("Pose Prediction", frame)

        # Exit on pressing 'q'
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()


In [8]:
def main():
    while True:
        choice = input("Choose an option: (1) Provide an image file (2) Real-time pose detection (3) Exit: ")

        if choice == "1":
            image_path = input("Enter the path to the image file: ")
            if os.path.exists(image_path):
                predicted_class, confidence, corrections = predict_pose_with_correction(image_path)
                print(f"Predicted Pose: {predicted_class}, Confidence: {confidence:.2f}%")
                if corrections:
                    print("Corrections Needed:")
                    for correction in corrections:
                        print(f"- {correction}")
                else:
                    print("No corrections needed.")
            else:
                print("File does not exist. Please check the path.")
        elif choice == "2":
            print("Starting real-time pose detection...")
            real_time_pose_prediction()
        elif choice == "3":
            print("Exiting the program. Goodbye!")
            break
        else:
            print("Invalid choice. Please choose again.")

# Run the main application
if __name__ == "__main__":
    main()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 180ms/step
Predicted Pose: goddess, Confidence: 96.02%
No corrections needed.
File does not exist. Please check the path.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
Predicted Pose: goddess, Confidence: 90.08%
No corrections needed.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
Predicted Pose: goddess, Confidence: 90.08%
No corrections needed.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
Predicted Pose: tree, Confidence: 99.99%
Corrections Needed:
- Image is too bright. Reduce brightness.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
Predicted Pose: warrior2, Confidence: 90.44%
Corrections Needed:
- Image is too bright. Reduce brightness.
Starting real-time pose detection...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━